# Google Colab: Drive'daki ZIP'i çıkartma
Bu not defteri, Google Drive'da bulunan büyük ZIP arşivini güvenle çıkartmak için gerekli adımları içerir. Adımları sırayla çalıştırın; ZIP yolu ve hedef klasörü size göre güncelleyin.

In [ ]:
# 1) Gerekli kütüphaneler ve Drive bağlama
try:
    from pathlib import Path
    import os, subprocess
    print("Libraries imported.")
except Exception as e:
    print("Import error:", e)
    raise

try:
    from google.colab import drive  # type: ignore
    print("Drive bağlanıyor...")
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print("Colab Drive modülü yok veya bağlanamadı:", e)

In [ ]:
# 2) ZIP ve hedef klasörleri tanımla (çıktı Drive'da kalacak)
from pathlib import Path
ZIP_INPUT = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn.zip')
# Çıktı Drive'da; ZIP ile aynı klasöre extracted alt-klasörü oluştur
EXTRACT_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn_extracted')
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

print('ZIP_INPUT =', ZIP_INPUT)
print('EXTRACT_DIR =', EXTRACT_DIR)
print('ZIP mevcut mu?', ZIP_INPUT.exists())
print('\nNot: Çıktı Drive\'da kalacak. Extraction sırasında network hızına bağlı olabilir.')

In [ ]:
# 3) 7-Zip (p7zip) ve unrar kur; optimized 110 GB extraction
!apt-get update -y >/dev/null 2>&1 && apt-get install -y p7zip-full unrar >/dev/null 2>&1

import subprocess
from pathlib import Path
import time

if ZIP_INPUT.exists():
    ext = ZIP_INPUT.suffix.lower()
    zip_size_gb = ZIP_INPUT.stat().st_size / (1024**3)
    print(f"ZIP boyutu: {zip_size_gb:.1f} GB")
    print(f"Hedef: {EXTRACT_DIR}")
    print("Başlıyor...")
    start = time.time()
    
    if ext == '.zip':
        # Devam edebilir (-aos), ilerleme göster (-bb1), multi-thread (-mmt=on)
        cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y -aos -bb1 -mmt=on'
        print('ZIP tespit edildi, 7z ile çıkarılıyor (ilerleme gösterilecek)...')
        print('CMD:', cmd)
        rc = subprocess.call(cmd, shell=True)
        elapsed = time.time() - start
        hours = elapsed / 3600
        print(f'\n7z exit code: {rc}')
        print(f'Toplam süre: {hours:.2f} saat ({elapsed/60:.1f} dakika)')
        
        if rc != 0:
            print('7z başarısız; unzip ile denenecek...')
            unzip_cmd = f'unzip -o "{ZIP_INPUT}" -d "{EXTRACT_DIR}"'
            print('CMD:', unzip_cmd)
            rc2 = subprocess.call(unzip_cmd, shell=True)
            print('unzip exit code:', rc2)
    elif ext == '.rar':
        print('RAR tespit edildi, unrar ile çıkarılıyor...')
        cmd = f'unrar x -o+ "{ZIP_INPUT}" "{EXTRACT_DIR}"'
        print('CMD:', cmd)
        rc = subprocess.call(cmd, shell=True)
        elapsed = time.time() - start
        hours = elapsed / 3600
        print(f'\nunrar exit code: {rc}')
        print(f'Toplam süre: {hours:.2f} saat ({elapsed/60:.1f} dakika)')
        
        if rc != 0:
            print('unrar başarısız; 7z ile denenecek...')
            cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y -aos -bb1 -mmt=on'
            print('CMD:', cmd)
            rc2 = subprocess.call(cmd, shell=True)
            print('7z exit code:', rc2)
    else:
        print(f'Bilinmeyen uzantı: {ext}. 7z ile denenecek...')
        cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y -aos -bb1 -mmt=on'
        print('CMD:', cmd)
        rc = subprocess.call(cmd, shell=True)
        elapsed = time.time() - start
        hours = elapsed / 3600
        print(f'7z exit code: {rc}, süre: {hours:.2f} saat')
else:
    print('ZIP_INPUT bulunamadı; Hücre 2\'deki yolu kontrol edin.')

In [ ]:
# 4) Çıkartılan içeriği doğrula ve örnek listeleme
import subprocess
print('Hızlı boyut özeti:')
subprocess.call(f"du -sh '{EXTRACT_DIR}'", shell=True)
subprocess.call(f"du -sh '{EXTRACT_DIR}'/* 2>/dev/null | sed -n '1,20p'", shell=True)

# İlk birkaç dosya
subprocess.call(f"find '{EXTRACT_DIR}' -maxdepth 2 -type f | head -n 20", shell=True)

In [ ]:
# 5) Drive dizininde PNG sayımı (rekürsif)
from pathlib import Path
import os
import math

# Bu yolu kendi Drive klasörüne göre ayarlayın:
DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/images')
RECURSIVE = True  # True: tüm alt klasörler dahil, False: sadece bu klasör

if not DIR.exists():
    print('Dizin yok:', DIR)
else:
    if RECURSIVE:
        files = [p for p in DIR.rglob('*') if p.is_file() and p.suffix.lower() == '.png']
    else:
        files = [p for p in DIR.iterdir() if p.is_file() and p.suffix.lower() == '.png']
    count = len(files)
    size_bytes = 0
    try:
        size_bytes = sum(p.stat().st_size for p in files)
    except Exception as e:
        print('Boyut hesaplama hatası:', e)
    size_gb = round(size_bytes / (1024**3), 2)
    print('Klasör:', DIR)
    print('PNG sayısı:', count)
    print('Toplam boyut (GB):', size_gb)

    # Örnek ilk 20 dosya
    for p in files[:20]:
        print(str(p))

In [ ]:
# 6) Tanı: Drive ve extract klasörlerinde PNG sayımı ve içerik kontrolü
from pathlib import Path
import os

# Drive'daki varsayılan yol (gerektiğinde güncelleyin):
DRIVE_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn/images')
EXTRACT_IMAGES = Path('/content/ds2_complete_tmn/images')
EXTS = {'.png'}  # gerekirse {'.png', '.jpg', '.jpeg'}

def count_files(dir_path: Path, recursive=True, exts=EXTS):
    if not dir_path.exists():
        return 0, []
    if recursive:
        files = [p for p in dir_path.rglob('*') if p.is_file() and p.suffix.lower() in exts]
    else:
        files = [p for p in dir_path.iterdir() if p.is_file() and p.suffix.lower() in exts]
    return len(files), files

def list_children(dir_path: Path, max_items=20):
    try:
        if dir_path.exists():
            print('İçerik (ilk', max_items, 'öğe):', dir_path)
            for p in list(dir_path.iterdir())[:max_items]:
                print(('DIR ' if p.is_dir() else 'FILE'), str(p))
        else:
            print('Klasör yok:', dir_path)
    except Exception as e:
        print('Listeleme hatası:', e)

print('--- DRIVE DIZINİ ---')
list_children(DRIVE_DIR.parent)
cnt_drive, files_drive = count_files(DRIVE_DIR, recursive=True)
print('Drive klasöründe PNG sayısı:', cnt_drive)
for p in files_drive[:20]:
    print(str(p))

print('\n--- EXTRACT DIZINI ---')
list_children(EXTRACT_IMAGES.parent)
cnt_ext, files_ext = count_files(EXTRACT_IMAGES, recursive=True)
print('Extract klasöründe PNG sayısı:', cnt_ext)
for p in files_ext[:20]:
    print(str(p))

print('\nNot: Drive klasörünüzde dosyalar ZIP içindeyse önce Hücre 3 ile extract edin ve EXTRACT_IMAGES altında kontrol edin. Gerekirse EXTS = {'.png', '.jpg', '.jpeg'} yapabilirsiniz.' )

In [ ]:
# 7) Gelişmiş sayım: Python + shell (case-insensitive), içerik listeleme
from pathlib import Path
import subprocess, os

# DİZİNİ BURADA AYARLAYIN:
DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn/images')  # gerekirse tek 'ds2_complete_tmn/images'
print('DIR =', DIR)
print('Exists?', DIR.exists())

# Python tarafı (case-insensitive)
def count_images_py(dir_path: Path, exts=('.png', '.jpg', '.jpeg')):
    if not dir_path.exists():
        return 0, []
    examples = []
    cnt = 0
    for p in dir_path.rglob('*'):
        try:
            if p.is_file():
                if p.suffix.lower() in exts:
                    cnt += 1
                    if len(examples) < 20:
                        examples.append(str(p))
        except Exception as e:
            pass
    return cnt, examples

cnt_py, examples_py = count_images_py(DIR)
print('Python count (png/jpg/jpeg):', cnt_py)
for p in examples_py:
    print(p)

# Shell tarafı (case-insensitive)
find_cmd = f"find '{DIR}' -type f \\( -iname '*.png' -o -iname '*.jpg' -o -iname '*.jpeg' \\) | wc -l"
print('Shell count cmd:', find_cmd)
try:
    out = subprocess.check_output(find_cmd, shell=True).decode().strip()
    print('Shell count:', out)
except Exception as e:
    print('Shell count error:', e)

# İçerik listeleme (ilk 40 satır)
print('\nListing:', DIR)
subprocess.call(f"ls -lah '{DIR}' | sed -n '1,40p'", shell=True)

print("\nNotlar:")
print("- Eğer klasör MyDrive yerine 'Shared drives' altındaysa yolu '/content/drive/Shared drives/...' olarak ayarlayın.")
print("- Çift 'ds2_complete_tmn' segmenti varsa doğru yolu seçin. Üst klasörü hücre 6 ile kontrol edebilirsiniz.")